In [21]:
# 2020-03-26:
# 1) Added trip distance corrections
# 2) using operatorcallsign instead of owner
# TODO:
# Explain where does airport come from
# Make sure flights that depart the day before are divided?

import requests
import time
import geopy.distance
import datetime  # To calculate today / yesterday UNIX timestamp
import math  # To see if not a number, NaN
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'  # required by the server
import pandas as pd
import numpy as np

getwd = os.getcwd()
if 'pedro' in getwd:
    dirpath = "../data/"
    sqlpath = '../website/db/ghgflights.db'
else:
    # This is for when running on the server on public_html
    # dirpath = os.getcwd()+"/public_html/ghgflights/data/"
    # sqlpath = os.getcwd()+"/public_html/ghgflights/db/ghgflights.db"
    # This is for when running on the server on /ghgflights
    parpath = os.path.dirname(getwd)
    dirpath = parpath+"/ghgflights/data/"
    sqlpath = parpath+"/ghgflights/db/ghgflights.db"

# opensky-network credentials
username = 'pdespouy'
password = '123456Qw'

# In case we want to use a sample of aircrafts in our local machine
ISTEST = True

# number of days with data
# a day is added as buffer
# max number of days is ~30
# 26 days works fine
DAYS = 26
DAYS += 1
if DAYS > 29:
    DAYS = 29

# dates variables
# NOTE: flights api just updates at midnight
end   = int(time.time())
begin = int(end - ((DAYS)*24*3600))  # 1 hour = 60 * 60

In [22]:
# Opensky aircrafts are labeled as ICAO24 (e.g. c06151), 
# but ICAO fuel-aircraft mapping has aircrafts models in IATA, e.g. B737 is 737 

# We download mapping (aircraftDatabase.csv) from 
# https://opensky-network.org/datasets/metadata/
# And make a lean version of it,
# from only aircrafts with icao codes

aircrafts = pd.read_csv(dirpath+'aircrafts_clean.csv', encoding='latin1')
aircrafts.head()

,airline_name,iata_airline,icao_airline,operatorcallsign,icao24,registration,manufacturericao,manufacturername,model,typecode,icaoaircrafttype,operator,operatoricao,operatoriata,owner,registered,icao,iata,model_y
0,Air Canada,AC,ACA,AIR CANADA,c03329,C-FTJS,AIRBUS,Airbus Industrie,A320-211 (Airbus),A320,L2J,NaN,ACA,NaN,Air Canada,1991-12-02,A320,320,Airbus A320
1,Air Canada,AC,ACA,AIR CANADA,c02eb9,C-FRSA,BOEING,The Boeing Company,787-9 (Boeing),B789,L2J,NaN,ACA,NaN,Air Canada,2017-01-05,B789,789,Boeing 787-9
2,Air Canada,AC,ACA,AIR CANADA,c0172e,C-FIUF,BOEING,The Boeing Company,777-233LR (Boeing),B77L,L2J,NaN,ACA,NaN,Air Canada,2007-08-07,B77L,77X,Boeing 777-200 Freighter
3,Air Canada,AC,ACA,AIR CANADA,c030fd,C-FSOI,NaN,The Boeing Company,737-8 (Boeing),NaN,NaN,Air Canada,ACA,AC,Air Canada,2018-06-23,B738,738,NaN
4,Air Canada,AC,ACA,AIR CANADA,c030a6,C-FSKZ,NaN,The Boeing Company,737-8 (Boeing),NaN,NaN,Air Canada,ACA,AC,Air Canada,2018-05-02,B738,738,NaN


In [23]:
aircrafts = aircrafts[['icao24','manufacturername','model','icao','iata','operatorcallsign']]
aircrafts.operatorcallsign.unique()

array(['AIR CANADA', 'JAZZ', 'CREE', 'AIR NORTH', 'DISCOVERY', 'TRANSAT',
       'AIRSPRINT', 'ALKAN AIR', 'PALLISER', 'BEARSKIN', 'CANADIAN',
       'EMPRESS', 'CARGOJET', 'GLACIER', 'COUGAR', 'FLAIR', 'HELIJET',
       'BOREK AIR', 'FLIGHTCRAFT', 'MORNINGSTAR', 'NORTHWRIGHT',
       'THUNDERBIRD', 'POLARIS', 'PASCO', 'PORTER AIR', 'PROPAIR',
       'SKYLINK', 'SUNWING', 'ATHABASKA', 'WESTJET'], dtype=object)

In [24]:
# There are duplicates because aircrafts with 
# airline_name 'Air Canada Express' are also under airline_name 'Jazz Air', 
# but with operatorcallsign JAZZ
if (ISTEST):
    aircrafts = aircrafts[aircrafts.operatorcallsign.isin(["WESTJET"])]
print('number of aircrafts:', len(aircrafts))
aircrafts.drop_duplicates(inplace=True)
print('number of aircrafts:', len(aircrafts))
aircrafts.head()

number of aircrafts: 129
number of aircrafts: 129


,icao24,manufacturername,model,icao,iata,operatorcallsign
885,c03bf5,Boeing,737-7CT (Boeing),B737,73G,WESTJET
886,c07fbe,Boeing,737-7CT (Boeing),B737,73G,WESTJET
887,c07fb9,Boeing,737-7CT (Boeing),B737,73G,WESTJET
888,c00734,The Boeing Company,737-8CT (Boeing),B738,738,WESTJET
889,c07923,Boeing,737-76N (Boeing),B737,73G,WESTJET


In [25]:
aircrafts = aircrafts.rename(columns={'operatorcallsign':'owner'}, errors='ignore')

In [26]:
# For testing purposes, we only take one aircraft
test1 = ['c0172e', 'c02eb9', 'c030fd', 'c030a6', 'c0315c', 'c05f0c', 'c03150', 'c05efa', 'c01b6c']
test2 = ['c08335']
if (ISTEST) :
    aircrafts = aircrafts[aircrafts.icao24.isin(test2)]
aircrafts

,icao24,manufacturername,model,icao,iata,owner
951,c08335,The Boeing Company,737-8CT (Boeing),B738,738,WESTJET


In [30]:
# We pull JSON from API and convert to a pandas dataframe
# Doc: https://opensky-network.org/apidoc/rest.html#departures-by-airport
df_array = []
path = '@opensky-network.org/api/flights/aircraft?icao24='
for index, row in aircrafts.iterrows():
    a = row['icao24'].lower()
    print(a)
    d = 'https://'+username+':'+password+path+a+'&begin='+str(begin)+'&end='+str(end)
    print(d)
    try:
        r = requests.get(d).json()
        # We need to transform the JSON to a pandas dataframe
        df_array.append(pd.json_normalize(r))
    except:
        continue
df_array

c08335
https://opensky-network.org/api/flights/aircraft?icao24=c08335&begin=1583532054&end=1585864854


[    icao24   firstSeen estDepartureAirport    lastSeen estArrivalAirport  \
 0   c08335  1585598911                CYXX  1585602420              CMF2   
 1   c08335  1585591120                CYEG  1585595436              CYXX   
 2   c08335  1585568104                CYHM  1585581179              CMF2   
 3   c08335  1585052914                CYHM  1585071845              CYHM   
 4   c08335  1584984519                CFQ7  1584995666              CYHM   
 ..     ...         ...                 ...         ...               ...   
 62  c08335  1583669823                CYHM  1583682388              None   
 63  c08335  1583611220                KFLL  1583640323              CYHM   
 64  c08335  1583591505                None  1583604715              KFLL   
 65  c08335  1583563002                KLAS  1583572105              CKA8   
 66  c08335  1583539979                CYXX  1583558807              KLAS   
 
     callsign  estDepartureAirportHorizDistance  \
 0   WSW204            

In [31]:
# We merge list of dataframes
ddf = pd.concat(df_array)
ddf.head()

,icao24,firstSeen,estDepartureAirport,lastSeen,estArrivalAirport,callsign,estDepartureAirportHorizDistance,estDepartureAirportVertDistance,estArrivalAirportHorizDistance,estArrivalAirportVertDistance,departureAirportCandidatesCount,arrivalAirportCandidatesCount
0,c08335,1585598911,CYXX,1585602420,CMF2,WSW204,1498.0,542.0,4422.0,1075.0,0,3
1,c08335,1585591120,CYEG,1585595436,CYXX,WSW267,9218.0,1600.0,9073.0,489.0,0,8
2,c08335,1585568104,CYHM,1585581179,CMF2,WSW101,1281.0,181.0,8110.0,733.0,0,4
3,c08335,1585052914,CYHM,1585071845,CYHM,WSW179,1719.0,67.0,2712.0,21.0,0,6
4,c08335,1584984519,CFQ7,1584995666,CYHM,WSW116,6021.0,1214.0,2444.0,51.0,1,6


In [32]:
# We want to select a few columns
ddf = ddf[['icao24','callsign','estDepartureAirport','estArrivalAirport','firstSeen','lastSeen']]
ddf.head()

,icao24,callsign,estDepartureAirport,estArrivalAirport,firstSeen,lastSeen
0,c08335,WSW204,CYXX,CMF2,1585598911,1585602420
1,c08335,WSW267,CYEG,CYXX,1585591120,1585595436
2,c08335,WSW101,CYHM,CMF2,1585568104,1585581179
3,c08335,WSW179,CYHM,CYHM,1585052914,1585071845
4,c08335,WSW116,CFQ7,CYHM,1584984519,1584995666


In [33]:
# We want to store in db flights with uknown airports
# We want to filter all flights that do not go to 'None'
# ddf = ddf[ddf['estArrivalAirport'].notnull()]
# ddf = ddf[ddf['estDepartureAirport'].notnull()]
# ddf.head()

In [34]:
# To remove duplicates from merging arrival and departure data frames
df = ddf.drop_duplicates()
print("Flights: ", len(df['icao24']))

Flights:  67


In [35]:
dfi = pd.merge(df, aircrafts, how='inner', on='icao24')
dfi.head()

,icao24,callsign,estDepartureAirport,estArrivalAirport,firstSeen,lastSeen,manufacturername,model,icao,iata,owner
0,c08335,WSW204,CYXX,CMF2,1585598911,1585602420,The Boeing Company,737-8CT (Boeing),B738,738,WESTJET
1,c08335,WSW267,CYEG,CYXX,1585591120,1585595436,The Boeing Company,737-8CT (Boeing),B738,738,WESTJET
2,c08335,WSW101,CYHM,CMF2,1585568104,1585581179,The Boeing Company,737-8CT (Boeing),B738,738,WESTJET
3,c08335,WSW179,CYHM,CYHM,1585052914,1585071845,The Boeing Company,737-8CT (Boeing),B738,738,WESTJET
4,c08335,WSW116,CFQ7,CYHM,1584984519,1584995666,The Boeing Company,737-8CT (Boeing),B738,738,WESTJET


In [36]:
fuel = pd.read_csv(dirpath+'fuel_consumption.csv')
# fuel[fuel['Code'] == '767']
fuel.head()

,Code,125,250,500,750,1000,1500,2000,2500,3000,...,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500
0,100,1296,2703,3788.0,5129.0,6427.0,8937.0,11373.0,13757.0,16104.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,141,1289,2754,3874.0,5258.0,6600.0,9199.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,142,1289,2754,3874.0,5258.0,6600.0,9199.0,11725.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,143,1324,2874,4105.0,5621.0,7100.0,9986.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,146,1289,2754,3874.0,5258.0,6600.0,9199.0,11725.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
def getInterval(d):
    if (d < 125):
        return [125, 125]
    elif ((d >= 125) and (d < 250)):
        return [125, 250]
    elif ((d >= 250) and (d < 500)):
        return [250, 500]
    elif ((d >= 500) and (d < 750)):
        return [500, 750]
    elif ((d >= 750) and (d < 1000)):
        return [750, 1000]
    elif ((d >= 1000) and (d < 1500)):
        return [1000, 1500]
    elif ((d >= 1500) and (d < 2000)):
        return [1500, 2000]
    elif ((d >= 2000) and (d < 2500)):
        return [2000, 2500]
    elif ((d >= 2500) and (d < 3000)):
        return [2500, 3000]
    elif ((d >= 3000) and (d < 3500)):
        return [3000, 3500]
    elif ((d >= 3500) and (d < 4000)):
        return [3500, 4000]
    elif ((d >= 4000) and (d < 4500)):
        return [4000, 4500]
    elif ((d >= 4500) and (d < 5000)):
        return [4500, 5000]
    elif ((d >= 5000) and (d < 5500)):
        return [5000, 5500]
    elif ((d >= 5500) and (d < 6000)):
        return [5500, 6000]
    elif ((d >= 6000) and (d < 6500)):
        return [6000, 6500]
    elif ((d >= 6500) and (d < 7000)):
        return [6500, 7000]
    elif ((d >= 7000) and (d < 7500)):
        return [7000, 7500]
    elif ((d >= 7500) and (d < 8000)):
        return [7500, 8000]
    else:
        return [8000, 8500]

In [38]:
# We want to get distances (in nm, calculated as great circle distance)
airports = pd.read_csv(dirpath+'airports.csv')
# airports = airports[['icao', 'latitude', 'longitude', 'city', 'region_code', 'country_code']]
airports = airports[['gps_code','latitude_deg','longitude_deg',
                     'municipality','iso_region','iso_country']]
airports = airports.rename(columns={
    'gps_code'     :'icao_airport',
    'latitude_deg' :'latitude',
    'longitude_deg':'longitude',
    'municipality' :'city',
    'iso_region'   :'region',
    'iso_country'  :'country'
})
# codeIataAirport;nameAirport;codeIso2Country;codeIcaoAirport;codeIataCity;latitudeAirport;longitudeAirport
# airports.columns.to_series().groupby(airports.dtypes).groups
# We remove rows with empty values in icao_airport
airports = airports[airports['icao_airport'].notnull()]
airports.head()

,icao_airport,latitude,longitude,city,region,country
0,00A,40.070801,-74.933601,Bensalem,US-PA,US
1,00AA,38.704022,-101.473911,Leoti,US-KS,US
2,00AK,59.949200,-151.695999,Anchor Point,US-AK,US
3,00AL,34.864799,-86.770302,Harvest,US-AL,US
5,00AS,34.942803,-97.818019,Alex,US-OK,US


In [39]:
# We will merge (outer join) the airport coordinates, and remove the following columns later:
# 'icao_x', 'icao_y', 'latitude_x', 'latitude_y', 'longitude_x', 'longitude_y'
dfi = pd.merge(dfi, airports, how='left', left_on='estDepartureAirport', right_on='icao_airport')
dfi = pd.merge(dfi, airports, how='left', left_on='estArrivalAirport', right_on='icao_airport')
# dfi.to_csv('data/test.csv')
dfi.head()

,icao24,callsign,estDepartureAirport,estArrivalAirport,firstSeen,lastSeen,manufacturername,model,icao,iata,...,longitude_x,city_x,region_x,country_x,icao_airport_y,latitude_y,longitude_y,city_y,region_y,country_y
0,c08335,WSW204,CYXX,CMF2,1585598911,1585602420,The Boeing Company,737-8CT (Boeing),B738,738,...,-122.361000,Abbotsford,CA-BC,CA,CMF2,53.184654,-113.738236,Edmonton,CA-AB,CA
1,c08335,WSW267,CYEG,CYXX,1585591120,1585595436,The Boeing Company,737-8CT (Boeing),B738,738,...,-113.580002,Edmonton,CA-AB,CA,CYXX,49.025299,-122.361000,Abbotsford,CA-BC,CA
2,c08335,WSW101,CYHM,CMF2,1585568104,1585581179,The Boeing Company,737-8CT (Boeing),B738,738,...,-79.934998,Hamilton,CA-ON,CA,CMF2,53.184654,-113.738236,Edmonton,CA-AB,CA
3,c08335,WSW179,CYHM,CYHM,1585052914,1585071845,The Boeing Company,737-8CT (Boeing),B738,738,...,-79.934998,Hamilton,CA-ON,CA,CYHM,43.173599,-79.934998,Hamilton,CA-ON,CA
4,c08335,WSW116,CFQ7,CYHM,1584984519,1584995666,The Boeing Company,737-8CT (Boeing),B738,738,...,-113.455002,Edmonton,CA-AB,CA,CYHM,43.173599,-79.934998,Hamilton,CA-ON,CA


In [40]:
def getCO2(model, nm):
    # We get intervals necessary to pull from ICAO table
    intervals = getInterval(nm)
    # print str(columns[0])
    f = fuel[fuel['Code'] == model][[str(intervals[0]), str(intervals[1])]]

    minx = f.iloc[0][0]
    maxx = f.iloc[0][1]
    if (minx == maxx):
        # Some corrections
        minx = 0
        intervals[0] = 0
        
    a = minx
    b = float(maxx - minx)  # on python 2.7, we need to declare variables as float to return us a float
    c = float(nm - intervals[0])
    d = float(intervals[1] - intervals[0])
    # kg_flight = minx + (float(maxx - minx)*float(nm - intervals[0])/float(intervals[1]-intervals[0]))
    kg_flight = a + (b*c/d)

    # constant representing the number of kg of CO2 produced by burning a kg of aviation fuel
    kg_to_c02 = 3.16
    # CO2 emissions in kg:
    co2 = kg_to_c02 * kg_flight
#     print "f", f
#     print "a+(b*c/d)"
#     print a, b, c, d
#     print model, kg_flight, co2
    return co2

In [41]:
# Test cell
# d = "CYYZ"
# a = "CYYC"
# coords_1 = (dfi[dfi['icao_x'] == d]['latitude_x'].values[0], dfi[dfi['icao_x'] == d]['longitude_x'].values[0])
# coords_2 = (dfi[dfi['icao_y'] == a]['latitude_y'].values[0], dfi[dfi['icao_y'] == a]['longitude_y'].values[0])
# nm = geopy.distance.distance(coords_1, coords_2).nm
# print(int(nm))

In [42]:
# As to convert timestamps to datetime
# needs to be here so it does not conflict with the previous import
from datetime import datetime

nm_array = []
co_array = []
dt_array = []
for index, row in dfi.iterrows():  # not sure why but we need to use index
    a = row['estArrivalAirport']
    d = row['estDepartureAirport']

    # We calculate the distance with longitude and latitude as input     
    if row['latitude_x'] and row['longitude_x']:
        coords_1 = (row['latitude_x'], row['longitude_x'])
    else:
        pass
    if row['latitude_y'] and row['longitude_y']:
        coords_2 = (row['latitude_y'], row['longitude_y'])
    else:
        pass
    # try/except so if function breaks, it does not stop
    try:
        nm = geopy.distance.distance(coords_1, coords_2).nm
        km = 1.852 * nm
        # We make trip distance corrections, since flights do not follow a straight line, page 8:
        # https://www.icao.int/environmental-protection/CarbonOffset/Documents/Methodology%20ICAO%20Carbon%20Calculator_v10-2017.pdf
        # Less than 550Km: +50 Km
        # Between 550Km and 5500Km: +100 Km
        # Above 5500Km: +125 Km
        if (km==0):
            pass
        elif (km < 550):
            km += 50
        elif (km < 5500):
            km += 100
        else:
            km += 125
        nm = km / 1.852
    except:
        nm = 0
    nm = int(nm)
    a = row['iata']
    if (nm == 0) or (a == 0):
        o = 0
    else:
        try:
            c = getCO2(a, nm)
            o = int(c)
        except:
            o = 0
    print(a,nm,o)
    dt = datetime.utcfromtimestamp(row['firstSeen'])
    dt = dt.strftime('%Y-%m-%d %H:%M:%S')
    nm_array.append(int(nm))
    co_array.append(o)
    dt_array.append(dt)

# We remove extra columns
# manufacturer is already in model column
columns = ['icao_airport_x', 'icao_airport_y', 
           'latitude_x'    , 'latitude_y'    , 
           'longitude_x'   , 'longitude_y'   ,
           'manufacturername']
df = dfi.drop(columns, axis=1)

df['distance']      = nm_array
df['co2']           = co_array
df['firstSeenDate'] = dt_array
df = df.sort_values(by=['distance'])
df.head()

738 464 14089
738 473 14217
738 1520 34057
738 0 0
738 1511 33898
738 1249 29077
738 1237 28856
738 0 0
738 0 0
738 863 21840
738 1078 25924
738 1078 25924
738 1835 39620
738 1022 24892
738 691 18465
738 473 14217
738 0 0
738 0 0
738 476 14260
738 1516 33987
738 0 0
738 874 22053
738 1835 39620
738 1818 39319
738 0 0
738 0 0
738 1727 37712
738 1727 37712
738 1078 25924
738 0 0
738 1523 34110
738 973 23965
738 0 0
738 0 0
738 0 0
738 916 22864
738 0 0
738 0 0
738 0 0
738 0 0
738 1078 25924
738 1078 25924
738 1727 37712
738 1673 36759
738 1650 36353
738 1727 37712
738 0 0
738 0 0
738 0 0
738 0 0
738 1835 39620
738 0 0
738 467 14132
738 447 13847
738 1835 39620
738 874 22053
738 863 21840
738 933 23192
738 939 23308
738 0 0
738 0 0
738 0 0
738 0 0
738 1078 25924
738 0 0
738 1184 27878
738 892 22400


,icao24,callsign,estDepartureAirport,estArrivalAirport,firstSeen,lastSeen,model,icao,iata,owner,city_x,region_x,country_x,city_y,region_y,country_y,distance,co2,firstSeenDate
33,c08335,WSW101,CYHM,None,1584274355,1584288125,737-8CT (Boeing),B738,738,WESTJET,Hamilton,CA-ON,CA,NaN,NaN,NaN,0,0,2020-03-15 12:12:35
24,c08335,WSW651,None,CYHM,1584477145,1584486248,737-8CT (Boeing),B738,738,WESTJET,NaN,NaN,NaN,Hamilton,CA-ON,CA,0,0,2020-03-17 20:32:25
32,c08335,WSW116,None,CYHM,1584293614,1584303989,737-8CT (Boeing),B738,738,WESTJET,NaN,NaN,NaN,Hamilton,CA-ON,CA,0,0,2020-03-15 17:33:34
34,c08335,WSW713,None,CYHM,1584231822,1584239636,737-8CT (Boeing),B738,738,WESTJET,NaN,NaN,NaN,Hamilton,CA-ON,CA,0,0,2020-03-15 00:23:42
36,c08335,WSW735,None,CYXU,1584204362,1584212129,737-8CT (Boeing),B738,738,WESTJET,NaN,NaN,NaN,London,CA-ON,CA,0,0,2020-03-14 16:46:02


In [43]:
print('Rows with distance > 0:', len(df[df['distance'] > 0]))
print('Total C02:', sum(df['co2']))
print('Count C02:', len(df['co2']))

Rows with distance > 0: 41
Total C02: 1130800
Count C02: 67


In [44]:
# We remove the whitespaces from callsign column
df['callsign'] = df['callsign'].str.strip()

In [45]:
# We remove commas from owner
df['owner'] = df['owner'].str.replace(',', '')

In [46]:
list(df)

['icao24',
 'callsign',
 'estDepartureAirport',
 'estArrivalAirport',
 'firstSeen',
 'lastSeen',
 'model',
 'icao',
 'iata',
 'owner',
 'city_x',
 'region_x',
 'country_x',
 'city_y',
 'region_y',
 'country_y',
 'distance',
 'co2',
 'firstSeenDate']

In [47]:
# We want the column names to be the same as the table column names
df.rename(columns={
    'estDepartureAirport':'fromAirport',
    'estArrivalAirport'  :'toAirport',
    
    'city_x'   :'fromCity',
    'region_x' :'fromRegion',
    'country_x':'fromCountry',
    
    'city_y'   :'toCity',
    'region_y' :'toRegion',
    'country_y':'toCountry'
}, inplace=True)

# We replace NaNs with '' in two columns
# NOTE: maybe we should do this in every
# column so conn.execute does not break
df['fromRegion'].fillna('', inplace=True)
df['toRegion'].fillna('', inplace=True)

In [48]:
list(df)

['icao24',
 'callsign',
 'fromAirport',
 'toAirport',
 'firstSeen',
 'lastSeen',
 'model',
 'icao',
 'iata',
 'owner',
 'fromCity',
 'fromRegion',
 'fromCountry',
 'toCity',
 'toRegion',
 'toCountry',
 'distance',
 'co2',
 'firstSeenDate']

In [49]:
df

,icao24,callsign,fromAirport,toAirport,firstSeen,lastSeen,model,icao,iata,owner,fromCity,fromRegion,fromCountry,toCity,toRegion,toCountry,distance,co2,firstSeenDate
33,c08335,WSW101,CYHM,None,1584274355,1584288125,737-8CT (Boeing),B738,738,WESTJET,Hamilton,CA-ON,CA,NaN,,NaN,0,0,2020-03-15 12:12:35
24,c08335,WSW651,None,CYHM,1584477145,1584486248,737-8CT (Boeing),B738,738,WESTJET,NaN,,NaN,Hamilton,CA-ON,CA,0,0,2020-03-17 20:32:25
32,c08335,WSW116,None,CYHM,1584293614,1584303989,737-8CT (Boeing),B738,738,WESTJET,NaN,,NaN,Hamilton,CA-ON,CA,0,0,2020-03-15 17:33:34
34,c08335,WSW713,None,CYHM,1584231822,1584239636,737-8CT (Boeing),B738,738,WESTJET,NaN,,NaN,Hamilton,CA-ON,CA,0,0,2020-03-15 00:23:42
36,c08335,WSW735,None,CYXU,1584204362,1584212129,737-8CT (Boeing),B738,738,WESTJET,NaN,,NaN,London,CA-ON,CA,0,0,2020-03-14 16:46:02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,c08335,WSW107,CYHM,CYCW,1584530181,1584547774,737-8CT (Boeing),B738,738,WESTJET,Hamilton,CA-ON,CA,Chilliwack,CA-BC,CA,1818,39319,2020-03-18 11:16:21
22,c08335,WSW106,CYXX,CYHM,1584552157,1584565829,737-8CT (Boeing),B738,738,WESTJET,Abbotsford,CA-BC,CA,Hamilton,CA-ON,CA,1835,39620,2020-03-18 17:22:37
50,c08335,WSW2206,CYXX,CYHM,1583899884,1583913205,737-8CT (Boeing),B738,738,WESTJET,Abbotsford,CA-BC,CA,Hamilton,CA-ON,CA,1835,39620,2020-03-11 04:11:24
12,c08335,WSW236,CYXX,CYHM,1584750786,1584765443,737-8CT (Boeing),B738,738,WESTJET,Abbotsford,CA-BC,CA,Hamilton,CA-ON,CA,1835,39620,2020-03-21 00:33:06


In [50]:
# NaNs are replaced with empty string
# so sql query does not break
df = df.replace(np.nan, '', regex=True)

In [51]:
# ignore so it does not break in case there is a duplicate
import sqlite3
conn = sqlite3.connect(sqlpath)
for index, row in df.iterrows():
    colnames = tuple(list(df))
    values   = tuple(list(row))
    query    = "INSERT OR IGNORE INTO flights {} VALUES {};".format(colnames, values)
    conn.execute(query)
conn.commit()
conn.close()

In [162]:
# NOTE: we have two columns with unique
# CREATE TABLE flights (id INTEGER PRIMARY KEY, icao24 TEXT, callsign TEXT, fromAirport TEXT, toAirport TEXT, 
#                       firstSeen INTEGER, lastSeen INTEGER, model TEXT, icao TEXT, iata TEXT, owner TEXT, 
#                       fromCity, fromRegion, fromCountry TEXT, toCity TEXT, toRegion TEXT, toCountry TEXT, 
#                       distance INTEGER, co2 INTEGER, firstSeenDate TEXT, UNIQUE(icao24, firstSeen));

In [163]:
# import subprocess
# subprocess.call("php /home/despfzpe/public_html/carbon/InsertToDB.php")